# Emotion Classification with Chain-of-Thought

This notebook trains an emotion classifier using DSPy's `ChainOfThought` module and optimizes it with GEPA.

In [1]:
from typing import Literal

import os
import pandas as pd

import dspy
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
GEMINI_KEY = os.getenv("GEMINI_API_KEY")

In [3]:
LABEL_MAPPING = {
    "0": "sadness",
    "1": "joy",
    "2": "love",
    "3": "anger",
    "4": "fear"
}

EMOTION_TYPES = list(LABEL_MAPPING.values())
EMOTION_TYPES

['sadness', 'joy', 'love', 'anger', 'fear']

## Load preprocessed data

In [4]:
# Load preprocessed train and validation data
df_train = pd.read_csv("data/train.csv")
df_val = pd.read_csv("data/val.csv")

print(f"Train size: {len(df_train)}")
print(f"Val size: {len(df_val)}")

df_train.head()

Train size: 100
Val size: 50


,user_message,emotion
0,i must say that i do feel better in myself and...,joy
1,i feel so privileged to have spent so much tim...,joy
2,i went to see the entrance examination results...,joy
3,i wish there was something i could do sitting ...,fear
4,i still can t shake the feeling of him loving ...,love


In [5]:
# Convert to DSPy format
train_data = df_train.to_dict(orient='records')
val_data = df_val.to_dict(orient='records')

dspy_train = [
    dspy.Example({
        "user_message": item["user_message"],
        "emotion": item["emotion"]
    }).with_inputs("user_message")
    for item in train_data
]

dspy_val = [
    dspy.Example({
        "user_message": item["user_message"],
        "emotion": item["emotion"]
    }).with_inputs("user_message")
    for item in val_data
]

len(dspy_train), len(dspy_val)

(100, 50)

## Configure DSPy

In [6]:
lm = dspy.LM(model="gemini/gemini-2.5-flash-lite")
dspy.configure(lm=lm)

## Define the emotion analyzer with Chain-of-Thought

In [7]:
EmotionTypes = Literal['sadness', 'joy', 'love', 'anger', 'fear']

class EmotionAnalyzer(dspy.Signature):
    """
    Determine the emotion of the user message.
    """
    user_message: str = dspy.InputField()
    emotion: EmotionTypes = dspy.OutputField()


class EmotionAnalyzerProgram(dspy.Module):
    def __init__(self):
        self.analyzer = dspy.ChainOfThought(EmotionAnalyzer)

    def forward(self, user_message: str) -> dspy.Prediction:
        emotion = self.analyzer(user_message=user_message)

        return emotion

In [8]:
em_analyzer = EmotionAnalyzerProgram()
em_analyzer("I am feeling good")

Prediction(
    reasoning='The user explicitly states "I am feeling good," which directly indicates a positive emotional state.',
    emotion='joy'
)

## Define metrics with reasoning feedback

In [9]:
def score_emotion(ground_truth: str, prediction: str) -> bool:
    return ground_truth == prediction


def emotion_metric(ground_truth, pred, trace=None, pred_name=None, pred_trace=None):
    # Handle both Example objects and strings
    gt_emotion = ground_truth.emotion if hasattr(ground_truth, 'emotion') else ground_truth
    pred_emotion = pred.emotion if hasattr(pred, 'emotion') else pred

    score_emotion_value = score_emotion(gt_emotion, pred_emotion)

    return score_emotion_value

def feedback_emotion(ground_truth, pred) -> tuple:
    # Handle both Example objects and strings
    gt_emotion = ground_truth.emotion if hasattr(ground_truth, 'emotion') else ground_truth
    pred_emotion = pred.emotion if hasattr(pred, 'emotion') else pred

    score = 1.0 if gt_emotion == pred_emotion else 0.0

    if gt_emotion == pred_emotion:
        feedback = f"You correctly classified the sentiment of the message as `{gt_emotion}`. This message is indeed `{gt_emotion}`."
    else:
        feedback = f"You incorrectly classified the sentiment of the message as `{pred_emotion}`. The correct sentiment is `{gt_emotion}`. Think about how you could have reasoned to get the correct sentiment label."
    return feedback, score

def metric_with_feedback(ground_truth, pred, trace=None, pred_name=None, pred_trace=None):
    # Handle both Example objects and strings
    gt_emotion = ground_truth.emotion if hasattr(ground_truth, 'emotion') else ground_truth
    pred_emotion = pred.emotion if hasattr(pred, 'emotion') else pred
    
    # Extract reasoning trace if available (from ChainOfThought)
    reasoning = getattr(pred, 'rationale', None) or getattr(pred, 'reasoning', None)
    
    score = 1.0 if gt_emotion == pred_emotion else 0.0
    
    if gt_emotion == pred_emotion:
        feedback = f"✓ Correct classification: `{gt_emotion}`"
        if reasoning:
            feedback += f"\n\nYour reasoning: {reasoning}"
            feedback += f"\n\nThis reasoning successfully led to the correct emotion. Continue using similar analytical approaches."
    else:
        feedback = f"✗ Incorrect: predicted `{pred_emotion}`, actual `{gt_emotion}`"
        if reasoning:
            feedback += f"\n\nYour reasoning was:\n{reasoning}"
            feedback += f"\n\nReflect on why this reasoning led to `{pred_emotion}` instead of `{gt_emotion}`. Consider:"
            feedback += f"\n- What specific words or context signal `{gt_emotion}` rather than `{pred_emotion}`?"
            feedback += f"\n- Did you focus on the right emotional indicators in the message?"
            feedback += f"\n- Were there subtle contextual clues you might have missed?"
        else:
            feedback += f"\n\nConsider what specific linguistic cues and context indicate `{gt_emotion}`."
    
    return dspy.Prediction(score=score, feedback=feedback)

## Baseline evaluation

In [10]:
evaluate = dspy.Evaluate(
    devset=dspy_val,
    metric=emotion_metric,
    num_threads=4,
    display_table=True,
    display_progress=True
)

evaluate(em_analyzer)

Average Metric: 24.00 / 50 (48.0%): 100%|██████████| 50/50 [00:13<00:00,  3.82it/s]

2025/10/19 10:00:26 INFO dspy.evaluate.evaluate: Average Metric: 24 / 50 (48.0%)


,user_message,example_emotion,reasoning,pred_emotion,emotion_metric
0,i only find out that they are looking and feeling complacent just ...,joy,The user expresses frustration and helplessness due to limited acc...,anger,
1,i can feel that my hopes have not been in vain she said,sadness,"The user's message expresses a feeling of hope being realized, whi...",joy,
2,i know how much work goes into the creation and i feel the author ...,joy,The user expresses an understanding of the effort involved in crea...,joy,✔️ [True]
3,i feel shamed that i hoped for one last christmas because i know s...,sadness,The user expresses shame for having a hope that is contrasted with...,sadness,✔️ [True]
4,i was feeling very bah humbugish coming out of this year s thanksg...,joy,"The user expresses negative feelings about the holiday season, spe...",sadness,
5,i am already feeling so tortured for having to wait for the result...,anger,"The user explicitly states they are feeling ""tortured"" and mention...",sadness,
6,i realize that i sound a little overdramatic when i say that but i...,sadness,The user expresses frustration and disappointment that their messa...,anger,
7,i won t feel like the jolly green giant while clothes shopping,joy,The user is expressing a negative sentiment about clothes shopping...,sadness,
8,i am frankly surprised that you consider the minds of the quorum m...,fear,The user expresses surprise and disagreement with the assessment o...,anger,
9,i mean i am happy for others but how can a person feel ok with som...,joy,"The user explicitly states they are ""happy for others"" but contras...",sadness,


EvaluationResult(score=48.0, results=<list of 50 results>)

## Optimize with GEPA

In [11]:
from dspy import GEPA

optimizer = GEPA(
    metric=metric_with_feedback,
    auto="light",
    num_threads=20,
    track_stats=True,
    use_merge=False,
    reflection_lm=dspy.LM(model="gemini/gemini-2.5-flash", temperature=1.0, max_tokens=10000, api_key=GEMINI_KEY)
)

In [12]:
optimized_program = optimizer.compile(
    em_analyzer,
    trainset=dspy_train,
    valset=dspy_val,
)

2025/10/19 10:00:41 INFO dspy.teleprompt.gepa.gepa: Running GEPA for approx 580 metric calls of the program. This amounts to 3.87 full evals on the train+val set.
2025/10/19 10:00:41 INFO dspy.teleprompt.gepa.gepa: Using 50 examples for tracking Pareto scores. You can consider using a smaller sample of the valset to allow GEPA to explore more diverse solutions within the same budget.
GEPA Optimization:   0%|          | 0/580 [00:00<?, ?rollouts/s]2025/10/19 10:00:42 INFO dspy.evaluate.evaluate: Average Metric: 24.0 / 50 (48.0%)
2025/10/19 10:00:42 INFO dspy.teleprompt.gepa.gepa: Iteration 0: Base program full valset score: 0.48
GEPA Optimization:   9%|▊         | 50/580 [00:00<00:03, 137.18rollouts/s]2025/10/19 10:00:42 INFO dspy.teleprompt.gepa.gepa: Iteration 1: Selected program 0 score: 0.48


Average Metric: 1.00 / 3 (33.3%): 100%|██████████| 3/3 [00:01<00:00,  1.99it/s]

2025/10/19 10:00:43 INFO dspy.evaluate.evaluate: Average Metric: 1.0 / 3 (33.3%)


2025/10/19 10:01:05 INFO dspy.teleprompt.gepa.gepa: Iteration 1: Proposed new text for analyzer.predict: Determine the dominant emotion of the user message.

When analyzing the user message, follow these detailed steps and considerations:

1.  **Identify Explicit Emotion Words:** Give high priority to words in the message that directly state or describe emotions (e.g., "embarrassed," "humiliated," "happy," "angry," "sad," "fearful"). These are strong indicators.

2.  **Analyze Contextual Information:** Carefully examine the surrounding details, the specific situation described, and any implied meanings. The broader context can significantly influence the interpretation of explicit emotions or reveal emotions that are not directly stated. For example, the context of being an "attack victim" amplifies negative feelings of shame and distress.

3.  **Infer the User's Perspective and Stance (User-Centric Analysis):**
    *   Crucially, consider *how the user feels about the situation, obser

Average Metric: 0.00 / 3 (0.0%): 100%|██████████| 3/3 [00:01<00:00,  2.26it/s]

...

2025/10/19 10:06:36 INFO dspy.evaluate.evaluate: Average Metric: 1.0 / 3 (33.3%)


2025/10/19 10:07:15 INFO dspy.teleprompt.gepa.gepa: Iteration 12: Proposed new text for analyzer.predict: Determine the dominant emotion of the user message.

When analyzing the user message, follow these detailed steps and considerations:

1.  **Identify Explicit Emotion Words:** Give high priority to words in the message that directly state or describe emotions (e.g., "embarrassed," "humiliated," "happy," "angry," "sad," "fearful"). These are strong indicators. However, be aware that explicit negative emotional language can sometimes describe a *secondary* emotion or a rhetorical device, and the *dominant* emotion might lie deeper, especially when the user is making an observation or expressing an achievement.

2.  **Analyze Contextual Information:** Carefully examine the surrounding details, the specific situation described, and any implied meanings. The broader context can significantly influence the interpretation of explicit emotions or reveal emotions that are not directly state

Average Metric: 2.00 / 3 (66.7%): 100%|██████████| 3/3 [00:01<00:00,  2.99it/s] 

2025/10/19 10:07:37 INFO dspy.evaluate.evaluate: Average Metric: 2.0 / 3 (66.7%)


2025/10/19 10:07:55 INFO dspy.teleprompt.gepa.gepa: Iteration 13: Proposed new text for analyzer.predict: Determine the dominant emotion of the user message.

When analyzing the user message, follow these detailed steps and considerations:

1.  **Identify Explicit Emotion Words & Their Nuance:**
    *   Give high priority to words in the message that directly state or describe emotions (e.g., "embarrassed," "humiliated," "happy," "angry," "sad," "fearful," "regretful"). These are strong indicators.
    *   Pay close attention to the *intensity* and *specific meaning* of these words within the context. For instance, "passionate" often indicates strong enthusiasm, excitement, or deep interest in an activity, event, or item, which typically aligns with `joy` (pleasure, excitement, enthusiasm) rather than `love` (a deeper, enduring affection or strong emotional bond towards a person, deeply cherished concept, or profound appreciation).
    *   Crucially, when the word "love" is explicitly 

## Inspect optimized prompts

In [13]:
for name, pred in optimized_program.named_predictors():
    print("================================")
    print(f"Predictor: {name}")
    print("================================")
    print("Prompt:")
    print(pred.signature.instructions)
    print("*********************************")

Predictor: analyzer.predict
Prompt:
Determine the dominant emotion of the user message.

When analyzing the user message, follow these detailed steps and considerations:

1.  **Identify Explicit Emotion Words:** Give high priority to words in the message that directly state or describe emotions (e.g., "embarrassed," "humiliated," "happy," "angry," "sad," "fearful"). These are strong indicators. However, be aware that explicit negative emotional language can sometimes describe a *secondary* emotion or a rhetorical device, and the *dominant* emotion might lie deeper, especially when the user is making an observation or expressing an achievement.

2.  **Analyze Contextual Information:** Carefully examine the surrounding details, the specific situation described, and any implied meanings. The broader context can significantly influence the interpretation of explicit emotions or reveal emotions that are not directly stated. For example, the context of being an "attack victim" amplifies nega

## Evaluate optimized program

In [27]:
# Get evaluation results
eval_results = evaluate(optimized_program)

Average Metric: 27.00 / 50 (54.0%): 100%|██████████| 50/50 [00:00<00:00, 106.23it/s]

2025/10/19 10:43:25 INFO dspy.evaluate.evaluate: Average Metric: 27 / 50 (54.0%)


,user_message,example_emotion,reasoning,pred_emotion,emotion_metric
0,i only find out that they are looking and feeling complacent just ...,joy,The user expresses a situation where they discovered something neg...,anger,
1,i can feel that my hopes have not been in vain she said,sadness,"The user is relaying a statement made by someone else: ""i can feel...",joy,
2,i know how much work goes into the creation and i feel the author ...,joy,The user expresses an understanding of the effort involved in crea...,love,
3,i feel shamed that i hoped for one last christmas because i know s...,sadness,"The user explicitly states ""i feel shamed"". Shame is a form of sad...",sadness,✔️ [True]
4,i was feeling very bah humbugish coming out of this year s thanksg...,joy,The user expresses a negative sentiment towards the commercial asp...,anger,
5,i am already feeling so tortured for having to wait for the result...,anger,"The user expresses feeling ""tortured"" due to having to wait for re...",anger,✔️ [True]
6,i realize that i sound a little overdramatic when i say that but i...,sadness,The user is expressing frustration and a sense of being misunderst...,anger,
7,i won t feel like the jolly green giant while clothes shopping,joy,"The user expresses a desire to not feel like the ""jolly green gian...",sadness,
8,i am frankly surprised that you consider the minds of the quorum m...,fear,"The user expresses surprise (""frankly surprised"") at two points: t...",anger,
9,i mean i am happy for others but how can a person feel ok with som...,joy,"The user explicitly states ""i am happy for others"" which indicates...",sadness,


In [30]:
# Extract the results
results = eval_results['results']

# Create dataframe
error_data = []
for example, prediction, is_correct in results:
    error_data.append({
        'user_message': example.user_message,
        'actual': example.emotion,
        'predicted': prediction.emotion,
        'correct': is_correct,
        'reasoning': prediction.reasoning if hasattr(prediction, 'reasoning') else None
    })

df_errors = pd.DataFrame(error_data)

# Errors only
df_incorrect = df_errors[df_errors['correct'] == False]


df_errors

,user_message,actual,predicted,correct,reasoning
0,i only find out that they are looking and feeling complacent just ...,joy,anger,False,The user expresses a situation where they discovered something neg...
1,i can feel that my hopes have not been in vain she said,sadness,joy,False,"The user is relaying a statement made by someone else: ""i can feel..."
2,i know how much work goes into the creation and i feel the author ...,joy,love,False,The user expresses an understanding of the effort involved in crea...
3,i feel shamed that i hoped for one last christmas because i know s...,sadness,sadness,True,"The user explicitly states ""i feel shamed"". Shame is a form of sad..."
4,i was feeling very bah humbugish coming out of this year s thanksg...,joy,anger,False,The user expresses a negative sentiment towards the commercial asp...
5,i am already feeling so tortured for having to wait for the result...,anger,anger,True,"The user expresses feeling ""tortured"" due to having to wait for re..."
6,i realize that i sound a little overdramatic when i say that but i...,sadness,anger,False,The user is expressing frustration and a sense of being misunderst...
7,i won t feel like the jolly green giant while clothes shopping,joy,sadness,False,"The user expresses a desire to not feel like the ""jolly green gian..."
8,i am frankly surprised that you consider the minds of the quorum m...,fear,anger,False,"The user expresses surprise (""frankly surprised"") at two points: t..."
9,i mean i am happy for others but how can a person feel ok with som...,joy,sadness,False,"The user explicitly states ""i am happy for others"" which indicates..."


In [32]:
df_incorrect.sort_values(by=["actual"])

,user_message,actual,predicted,correct,reasoning
15,i feel a bit dissatisfied,anger,sadness,False,"The user explicitly states ""i feel a bit dissatisfied"". Dissatisfa..."
39,i am feeling lots of movement now but gar is unsure whether he fee...,fear,sadness,False,"The user is describing a physical sensation (""feeling lots of move..."
8,i am frankly surprised that you consider the minds of the quorum m...,fear,anger,False,"The user expresses surprise (""frankly surprised"") at two points: t..."
10,i can not help but feel distraught about it,fear,sadness,False,"The user explicitly states ""i can not help but feel distraught abo..."
40,i guess my nephew feels like crap but the popular opinion is he ll...,joy,sadness,False,The user is expressing an observation about their nephew's emotion...
35,im with my boyfriend and friends i feel fine and genuinely happy b...,joy,sadness,False,"The user explicitly states they feel ""genuinely happy"" when with t..."
29,i rarely feel inspired and ready to write,joy,sadness,False,"The user states they ""rarely feel inspired and ready to write."" Th..."
44,i feel like i havent blogged in a super long time,joy,sadness,False,The user expresses a feeling of having not blogged for a long time...
0,i only find out that they are looking and feeling complacent just ...,joy,anger,False,The user expresses a situation where they discovered something neg...
7,i won t feel like the jolly green giant while clothes shopping,joy,sadness,False,"The user expresses a desire to not feel like the ""jolly green gian..."


In [33]:
optimized_program.save("optimized_emotion_classifier.json")